In [107]:
import pandas as pd
import numpy as np
import torch
from collections import Counter
import random
import scipy.stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import torchvision.transforms as transforms
import albumentations as A

In [108]:
"""labels:
0 - other
1 - head non-standard plane
2 - head standard plane
3 - femur non-standard plane
4 - femur standard plane
5 - femur non standard plane
6 - femur standard plane"""

'labels:\n0 - other\n1 - head non-standard plane\n2 - head standard plane\n3 - femur non-standard plane\n4 - femur standard plane\n5 - femur non standard plane\n6 - femur standard plane'

In [109]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_train_scaled_size.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_val_scaled_size.csv')
test = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/biometry_test_scaled_size.csv')

In [11]:
train_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_train.csv')
test_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_test.csv')

val_org = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_val.csv')

In [12]:
train['days'] = train_org['days']
test['days'] = test_org['days']
val['days'] = val_org['days']

In [14]:
train = pd.concat([train, test])

In [16]:
train.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/class_data/biometry_train_all.csv', index=False)
val.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/class_data/biometry_val_all.csv', index=False)

In [124]:
train = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_train_scaled_size_all.csv')
val = pd.read_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/biometry_val_scaled_size_all.csv')

In [122]:
long_vids = (train['video'].value_counts() > 3)
videos = train['video'].unique()
long_vids = videos[long_vids]
train_filtered = train[train['video'].isin(long_vids)]

In [123]:
train_filtered

,index,Class,video,measures,ps,frames_n,measure_scaled,days,frame_loc,height,width,height_org,width_org,measure_normalized
0,6_2_35,4,6_2,16.47,0.120130,51,0.618863,147,0.686275,86.493600,115.324800,720,960,0.457089
1,6_2_36,4,6_2,16.47,0.120130,51,0.618863,147,0.705882,86.493600,115.324800,720,960,0.457089
2,6_2_37,4,6_2,16.47,0.120130,51,0.618863,147,0.725490,86.493600,115.324800,720,960,0.457089
3,6_2_38,4,6_2,16.47,0.120130,51,0.618863,147,0.745098,86.493600,115.324800,720,960,0.457089
4,6_2_39,4,6_2,16.47,0.120130,51,0.618863,147,0.764706,86.493600,115.324800,720,960,0.457089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33702,490_3_248,6,490_3,6.10,0.157282,310,0.139177,225,0.800000,134.004264,178.672352,852,1136,0.131091
33703,490_3_249,6,490_3,6.10,0.157282,310,0.139177,225,0.803226,134.004264,178.672352,852,1136,0.131091
33704,490_3_250,6,490_3,6.10,0.157282,310,0.139177,225,0.806452,134.004264,178.672352,852,1136,0.131091
33705,490_3_251,6,490_3,6.10,0.157282,310,0.139177,225,0.809677,134.004264,178.672352,852,1136,0.131091


In [125]:
long_vids = (val['video'].value_counts() > 3)
videos = val['video'].unique()
long_vids = videos[long_vids]
val_filtered = val[val['video'].isin(long_vids)]

In [127]:
val_filtered.to_csv('/data/kpusteln/Fetal-RL/data_preparation/data_biometry/ete_model/biometry_scaled_ps/all/key_frame_data/biometry_val_all_filtered.csv', index=False)

In [130]:
def clamp(num, min_value, max_value):
   return max(min(num, max_value), min_value)

In [ ]:
class Video_Loader():
    def __init__(self, root, videos_path, ann_path, transform=None, target_transform=None, img_scaling = False):
        
        self.data_path = root
        self.ann_path = ann_path
        self.videos = pd.read_csv(videos_path)
        self.data = pd.read_csv(self.ann_path)
        self.img_scaling = img_scaling
        
        
    def _load_image(self, path):
        try:
            im = Image.open(path)
            im.convert('RGB')
        except:
            print("ERROR IMG LOADED: ", path)
            random_img = np.random.rand(224, 224, 3) * 255
            im = Image.fromarray(np.uint8(random_img))
        return im
    
    def __len__(self):
        return len(self.data)
    
    
    def _load_item(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        idb = self.data[data['index'] == index]
        frame_idx = idb[0]
        Class = torch.tensor(idb[1])
        video = idb[2]
        measure = torch.tensor(idb[3])
        ps = torch.tensor(idb[4])
        frames_n = torch.tensor(idb[5])
        measure_scaled = torch.tensor(idb[6], dtype=torch.float32)
        days_normalized = torch.tensor(idb[7], dtype=torch.float32)
        frame_loc = torch.tensor(idb[8], dtype=torch.float32)
        height = torch.tensor(idb[9])
        width = torch.tensor(idb[10])
        measure_normalized = torch.tensor(idb[13], dtype=torch.float32)
        images = self._load_image(self.data_path  + frame_idx + '.png')
        
        new_height = clamp(int(height*2.161191086437513), 0, 512)
        new_width = clamp(int(width*2.161191086437513), 0, 512)
        if self.img_scaling:
            padding = A.PadIfNeeded(min_height=512, min_width=512, border_mode=0, value=0, mask_value=0, always_apply=False, p=1.0)
            rescale = A.Resize(new_height, new_width)
            images = rescale(image = images)['image']
            images = padding(image=images)['image']
            images = np.expand_dims(images, 2)
            t = transforms.Compose([transforms.ToTensor(),])
            images = t(images)
        else:
            images = np.expand_dims(images, 2)
            t = transforms.Compose([transforms.ToTensor(),
            transforms.Resize((450, 600)),
            transforms.Pad((0, 0, 0, 150), fill = 0, padding_mode = 'constant'),
            transforms.Resize((512, 512)),
            transforms.Normalize(mean=0.1354949, std=0.18222201)])
            images = t(images)
        if self.transform is not None:
            images = self.transform(images)

        return images, Class, measure, ps, frames_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized
    
    
    def __getitem__(self, index):
        video = self.videos[index]
        batch = self.data[self.data['video'] == video].sample(8)
        frame_idx = batch['index']
        images = list()
        Classes = list()
        indexes = list()
        for i in frame_idx:
            image, Class, measure, ps, frame_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized = self._load_item(i)
            images.append(image)
            Classes.append(Class)
            indexes.append(index)
        return images, Classes, measure_scaled, ps, frame_n, measure_scaled, index, days_normalized, frame_loc, measure_normalized

In [160]:
def features_padding(features, max_length):
    padding_length = max_length - features.size(1)
    padded_seq = torch.nn.functional.pad(features, (0, 0, 0, padding_length, 0, 0), mode='constant', value=0)
    return padded_seq

In [199]:
padded_seq = features_padding(original_seq, 8)

In [183]:
import torch

def mask_sequence(padded_seq, org_seq_len, max_len):  
    
    # Define the padded sequence
    
    padded_seq = torch.randn(1, max_len, 1280)

    # Define the mask tensor
    mask = torch.zeros(padded_seq.size()[:2], dtype=torch.float32)

    # Set the non-padding elements to 1's
    mask[:, :org_seq_len] = 1

    # Expand the mask tensor to shape (1, 8, 1) and (1, 1, 8)
    mask_1 = mask.unsqueeze(1)
    mask_2 = mask.unsqueeze(2)

    # Compute the element-wise multiplication of the two mask tensors
    attention_mask = mask_1 * mask_2

    # The attention mask has shape (1, 8, 8)
    return attention_mask




In [185]:
mask = mask_sequence(original_seq, 4, 8)

In [186]:
attn_scores = torch.randn(1, 8, 8)

In [207]:
attn_scores.masked_fill(mask == 0, value=-1e9)

tensor([[[-6.0653e-01,  1.0171e+00,  1.2028e+00,  1.0150e+00, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 2.2674e-01,  2.0132e-01, -1.3626e-01, -4.3834e-01, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [ 1.7038e+00,  8.7594e-01,  1.0507e+00,  5.6731e-01, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-2.0116e-01,  6.3637e-01,  8.5214e-01,  8.1813e-01, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09],
         [-1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09, -1.0000e+09,
          -1.0000e+09, -1.0000e+09, -1.0000e+09]]]

In [ ]:
def avg_pool_mask(features, org_seq_len):
    features[:, org_seq_len:, :]
    return features

In [204]:
padded_seq[:, :3, :].shape

torch.Size([1, 3, 1280])